In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
sns.set_theme()

import pycountry
import pycountry_convert as pc

import folium

from locationHelper import LocationHelper
np.random.seed(42)
from datetime import datetime

In [2]:
RATE_BEER_URL = "./data/RateBeer/"
BEER_DATA_URL = "./data/BeerAdvocate/"

# PreProcessing
We do both for BeerAdvocate and RateBeer separately. In the end there we generate a preProcessingHelper.py that cleans the datasets and saves the preprocessed csv.

## Reviews / Ratings
We focus on the ratings and reviews as this is what we mainly need for our task. The reviews are a subset of the ratings. We have a lot more ratings than than reviews but the elements difference between ratings and reviews don't contain any specific rating like palette, taste, ...

In [4]:
def parse_beer_rating(rating, dropText = True):
    rating_data = {}
    for line in rating.split('\n'):
        key, value = line.split(':', 1)

        if dropText and key == "text":
            continue

        rating_data[key.strip()] = value.strip()
    return rating_data


with open(BEER_DATA_URL + 'test_ratings.txt', 'r') as file:
    content = file.read()

    # Split the content by blank lines into separate reviews
    review_blocks = content.strip().split('\n\n')

    # Parse each review and store in a list of dictionaries
    parsed_reviews = [parse_beer_rating(review) for review in review_blocks]

ratings_df_init = pd.DataFrame(parsed_reviews)
ratings_df = ratings_df_init.copy()

In [5]:
ratings_df

,beer_name,beer_id,brewery_name,brewery_id,style,abv,date,user_name,user_id,appearance,aroma,palate,taste,overall,rating,review
0,Régab,142544,Societe des Brasseries du Gabon (SOBRAGA),37262,Euro Pale Lager,4.5,1440064800,nmann08,nmann08.184925,3.25,2.75,3.25,2.75,3.0,2.88,True
1,Barelegs Brew,19590,Strangford Lough Brewing Company Ltd,10093,English Pale Ale,4.5,1235127600,StJamesGate,stjamesgate.163714,3.0,3.5,3.5,4.0,3.5,3.67,True
2,Barelegs Brew,19590,Strangford Lough Brewing Company Ltd,10093,English Pale Ale,4.5,1142247600,mdagnew,mdagnew.19527,4.0,3.5,3.5,4.0,3.5,3.73,True


In [ ]:
# convert boolean strings to bool
def booleanConverter(x): 
    x = x.lower()
    if x == "false":
        return False
    elif x == "true":
        return True
    else:
        return np.nan

ratings_df["review"] = ratings_df_init["review"].map(booleanConverter)

In [6]:
dtypes = {
    "beer_name": "str",
    "beer_id": "int",
    "brewery_name": "str",
    "brewery_id": "int",
    "style": "str",
    "abv": "float",
    "date": "int",
    "user_name": "str",
    "user_id": "str",
    "appearance": "float",
    "aroma": "float",
    "palate": "float",
    "taste": "float",
    "overall": "float",
    "rating": "float",
    "review": "bool"}

ratings_df = ratings_df.astype(dtypes)

In [7]:
# only reviews, therefore it makes sense to see other metrics than rating
reviews_df = ratings_df[ratings_df["review"]]

# for the ratings, only the rating attribute is of value
ratings_df.drop(["appearance", "aroma", "palate", "taste", "overall"], axis=1, inplace=True)

In [12]:
print("ratings_df_init memory usage: ", ratings_df_init.memory_usage(index=True).sum())
print("ratings_df memory usage: ", ratings_df.memory_usage(index=True).sum())
print("reviews_df memory usage: ", reviews_df.memory_usage(index=True).sum())

ratings_df_init memory usage:  1074308228
ratings_df memory usage:  679835724
reviews_df memory usage:  334056594


In [13]:
ratings_df.describe()

,beer_id,brewery_id,abv,date,rating
count,8.393032e+06,8.393032e+06,8.221727e+06,8.393032e+06,8.393032e+06
mean,6.675445e+04,9.129888e+03,7.330266e+00,1.375265e+09,3.882131e+00
std,6.481824e+04,1.237611e+04,2.459114e+00,8.678273e+07,6.205088e-01
min,3.000000e+00,1.000000e+00,1.000000e-02,8.407080e+08,1.000000e+00
25%,9.074000e+03,1.920000e+02,5.500000e+00,1.339322e+09,3.540000e+00
50%,5.226600e+04,1.142000e+03,6.900000e+00,1.395227e+09,4.000000e+00
75%,9.654800e+04,1.800600e+04,8.800000e+00,1.431684e+09,4.250000e+00
max,2.932960e+05,4.981500e+04,6.750000e+01,1.501582e+09,5.000000e+00


In [21]:
reviews_df[["rating", "appearance", "aroma", "palate", "taste", "overall"]].describe()

,rating,appearance,aroma,palate,taste,overall
count,2.589586e+06,2.557577e+06,2.557577e+06,2.557577e+06,2.557577e+06,2.557577e+06
mean,3.828833e+00,3.884245e+00,3.794767e+00,3.798682e+00,3.836279e+00,3.846197e+00
std,6.014400e-01,5.868417e-01,6.611576e-01,6.507799e-01,6.936974e-01,6.719210e-01
min,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
25%,3.530000e+00,3.500000e+00,3.500000e+00,3.500000e+00,3.500000e+00,3.500000e+00
50%,3.930000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00
75%,4.220000e+00,4.250000e+00,4.250000e+00,4.000000e+00,4.250000e+00,4.250000e+00
max,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00


In [50]:
# check if we can reduce the size
print(ratings_df["beer_id"].astype(int).max())
print(ratings_df["brewery_id"].astype(int).max())
print(ratings_df["date"].astype(int).max())
print(np.iinfo(np.int16))
print(np.iinfo(np.int32))
print(np.finfo(np.float16))

293296
49815
1501581600
Machine parameters for int16
---------------------------------------------------------------
min = -32768
max = 32767
---------------------------------------------------------------

Machine parameters for int32
---------------------------------------------------------------
min = -2147483648
max = 2147483647
---------------------------------------------------------------

Machine parameters for float16
---------------------------------------------------------------
precision =   3   resolution = 1.00040e-03
machep =    -10   eps =        9.76562e-04
negep =     -11   epsneg =     4.88281e-04
minexp =    -14   tiny =       6.10352e-05
maxexp =     16   max =        6.55040e+04
nexp =        5   min =        -max
smallest_normal = 6.10352e-05   smallest_subnormal = 5.96046e-08
---------------------------------------------------------------



In [59]:
# change floats to less precision as we only have steps of 0.25
dtypes = {
    "appearance": np.float16,
    "aroma": np.float16,
    "palate": np.float16,
    "taste": np.float16,
    "overall": np.float16,
    "rating": np.float16 # precision of rating is on two decimals
    }
reviews_df = reviews_df.astype(dtype=dtypes)
ratings_df = ratings_df.astype(dtype={"rating": np.float16})

print("ratings_df memory usage: ", ratings_df.memory_usage(index=True).sum())
print("reviews_df memory usage: ", reviews_df.memory_usage(index=True).sum())

ratings_df memory usage:  629477532
reviews_df memory usage:  240831498


In [ ]:
# corrupt columns for ratings_df -> no corrupt columns
relevant_columns = ["rating", "date", "user_id"]
nr_corrupt_cols = ratings_df.shape[0] - ratings_df[relevant_columns].isna().shape[0]
print(nr_corrupt_cols)

0


In [ ]:
# corrupt columns for reviews_df -> no corrupt columns
relevant_columns = ["rating", "appearance", "aroma", "palate", "taste", "overall", "date", "user_id"]
nr_corrupt_cols = reviews_df.shape[0] - reviews_df[relevant_columns].isna().shape[0]
print(nr_corrupt_cols)

0


### Date/Time

In [ ]:
def int_to_datetime(df: pd.DataFrame):
    df["date_object"] = df["date"].apply(datetime.fromtimestamp)

def datetime_to_month(df: pd.DataFrame):
    try:
        df["month"] = df["date_object"].apply(lambda x: x.month)
    except KeyError:
        print("first create a datetime object column!")

def datetime_to_year(df: pd.DataFrame):
    try:
        df["year"] = df["date_object"].apply(lambda x: x.year)
    except KeyError:
        print("first create a datetime object column!")


int_to_datetime(reviews_df)
reviews_df.head(10)

/Users/klusertim/miniforge3/envs/ada/lib/python3.11/site-packages/pandas/io/formats/format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


,beer_name,beer_id,brewery_name,brewery_id,style,abv,date,user_name,user_id,appearance,aroma,palate,taste,overall,rating,review,date_object
0,Régab,142544,Societe des Brasseries du Gabon (SOBRAGA),37262,Euro Pale Lager,4.5,<bound method Timestamp.date of Timestamp('201...,nmann08,nmann08.184925,3.25,2.75,3.25,2.75,3.00,2.880859,True,2015-08-20 12:00:00
1,Barelegs Brew,19590,Strangford Lough Brewing Company Ltd,10093,English Pale Ale,4.5,<bound method Timestamp.date of Timestamp('200...,StJamesGate,stjamesgate.163714,3.00,3.50,3.50,4.00,3.50,3.669922,True,2009-02-20 12:00:00
2,Barelegs Brew,19590,Strangford Lough Brewing Company Ltd,10093,English Pale Ale,4.5,<bound method Timestamp.date of Timestamp('200...,mdagnew,mdagnew.19527,4.00,3.50,3.50,4.00,3.50,3.730469,True,2006-03-13 12:00:00
3,Barelegs Brew,19590,Strangford Lough Brewing Company Ltd,10093,English Pale Ale,4.5,<bound method Timestamp.date of Timestamp('200...,helloloser12345,helloloser12345.10867,4.00,3.50,4.00,4.00,4.50,3.980469,True,2004-12-01 12:00:00
4,Barelegs Brew,19590,Strangford Lough Brewing Company Ltd,10093,English Pale Ale,4.5,<bound method Timestamp.date of Timestamp('200...,cypressbob,cypressbob.3708,4.00,4.00,4.00,4.00,4.00,4.000000,True,2004-08-30 12:00:00
12,Legbiter,19827,Strangford Lough Brewing Company Ltd,10093,English Pale Ale,4.8,<bound method Timestamp.date of Timestamp('201...,brewdude1982,brewdude1982.691169,4.25,4.50,3.25,3.75,3.75,3.910156,True,2013-02-13 12:00:00
13,Legbiter,19827,Strangford Lough Brewing Company Ltd,10093,English Pale Ale,4.8,<bound method Timestamp.date of Timestamp('201...,AgentMunky,agentmunky.409755,4.00,3.75,3.50,3.50,3.75,3.640625,True,2013-01-09 12:00:00
17,Legbiter,19827,Strangford Lough Brewing Company Ltd,10093,English Pale Ale,4.8,<bound method Timestamp.date of Timestamp('201...,ferrari29,ferrari29.320392,3.00,3.50,2.50,2.50,2.50,2.769531,True,2012-11-17 12:00:00
19,Legbiter,19827,Strangford Lough Brewing Company Ltd,10093,English Pale Ale,4.8,<bound method Timestamp.date of Timestamp('201...,OtherShoe2,othershoe2.233642,4.00,3.50,4.00,3.50,4.00,3.679688,True,2012-07-14 12:00:00
21,Legbiter,19827,Strangford Lough Brewing Company Ltd,10093,English Pale Ale,4.8,<bound method Timestamp.date of Timestamp('201...,brentk56,brentk56.6284,4.50,3.00,3.50,2.50,2.50,2.839844,True,2012-04-28 12:00:00


### With our preProcessingHelper

In [10]:
%load_ext autoreload
%autoreload 2
from preProcessingHelper import PreProcessRatings

pp = PreProcessRatings(platform="BeerAdvocate")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
start parsing the beer reviews for BeerAdvocate
finished parsing the beer reviews for BeerAdvocate


In [11]:
df = pp.get_dataframe(dataset="ratings", additinal_cols=["country_name", "country_code3", "state", "date_object", "month"])
df.save_csv("data/myData/allBeerAdvocate.csv")

Country svalbard and jan mayen islands could not be resolved with pycountry fuzzy search.
Try to modify it in polish_country_names()
Country aotearoa could not be resolved with pycountry fuzzy search.
Try to modify it in polish_country_names()
Country fiji islands could not be resolved with pycountry fuzzy search.
Try to modify it in polish_country_names()
Country heard and mcdonald islands could not be resolved with pycountry fuzzy search.
Try to modify it in polish_country_names()
Country south georgia and south sandwich islands could not be resolved with pycountry fuzzy search.
Try to modify it in polish_country_names()
From 8393032 reviews, 7892754 have a location (corresponding to the user) and 500278 do not have a location


In [14]:
df.to_csv("data/tims_data/allBeerAdvocate.csv")

In [31]:
pp.get_dataframe(dataset="ratings", additinal_cols=["date_object", "beer_id"]).head(3)

,rating,date_object,beer_id
0,2.88,2015-08-20 12:00:00,142544
1,3.67,2009-02-20 12:00:00,19590
2,3.73,2006-03-13 12:00:00,19590


In [32]:
pp.get_dataframe(dataset="reviews", additinal_cols=["month", "country_name"]).head(3)

From 39 reviews, 38 have a location (corresponding to the user) and 1 do not have a location


,rating,appearance,aroma,palate,taste,overall,month,country_name
0,2.88,3.25,2.75,3.25,2.75,3.0,8,United States
1,3.67,3.00,3.50,3.50,4.00,3.5,2,United States
2,3.73,4.00,3.50,3.50,4.00,3.5,3,United Kingdom
